In [1]:
# for notebook plotting
%matplotlib inline 

# load what we need
import numpy as np
import pandas as pd

In [2]:
# read in the results data sets
save_path = "/datadisk/whiteboxing/examples2/german/"
tc = 'tc' # true column
pc = 'pc' # pred column

def load_preds(filename):
    out = pd.read_csv(save_path + filename,
                      usecols=['true class', 'predicted class'],
                     )
    out.columns = [tc, pc]
    return(out)

def load_full(filename):
    out = pd.read_csv(save_path + filename)
    return(out)

rf_preds = load_preds('RandomForest_CHIRPS_rnst_demo123.csv')
ada1_preds = load_preds('AdaBoost1_CHIRPS_rnst_demo123.csv')
ada2_preds = load_preds('AdaBoost2_CHIRPS_rnst_demo123.csv')
gbt_preds = load_preds('GBM_CHIRPS_rnst_demo123.csv')

rf = load_full('RandomForest_CHIRPS_rnst_demo123.csv')

In [3]:
# add an agreement column to each
for pred in [rf_preds, ada1_preds, ada2_preds, gbt_preds]:
    pred['agree'] = pred[tc] == pred[pc]

In [14]:
# identify alignment and agreement
pred_align = rf_preds[pc] + ada1_preds[pc] + ada2_preds[pc] + gbt_preds[pc] # pd.Series type
alag = pd.concat([rf['instance_id'],
                  rf_preds[pc],
                  ada1_preds[pc],
                  ada2_preds[pc],
                  gbt_preds[pc],
                  rf_preds[tc],
                  pred_align],
                 ignore_index=True, axis=1)
alag.columns = ['id', 'rf_p', 'a1_p', 'a2_p', 'gbt_p', tc, 'align']
alag['categ'] = 'disagree'
alag.loc[(alag['align'] == 4) & (alag[tc] == 1), 'categ'] = 'agree_bad_right'
alag.loc[(alag['align'] == 4) & (alag[tc] == 0), 'categ'] = 'agree_bad_wrong'
alag.loc[(alag['align'] == 0) & (alag[tc] == 1), 'categ'] = 'agree_good_wrong'
alag.loc[(alag['align'] == 0) & (alag[tc] == 0), 'categ'] = 'agree_good_right'

In [ ]:
alag